In [1]:
pip install transformers

In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [3]:
from datasets import load_dataset
dataset = load_dataset("allenai/ai2_arc", 'ARC-Easy')
train = dataset['train']
test = dataset['test']
validation = dataset['validation']

Generating train split:   0%|          | 0/2251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2376 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/570 [00:00<?, ? examples/s]

In [10]:
from datasets import concatenate_datasets
final_dataset = concatenate_datasets([train, test, validation])

In [11]:
import pandas as pd
df = pd.DataFrame(final_dataset)

In [12]:
filtered_rows = df[df['answerKey'] == 'E']
filtered_rows

,id,question,choices,answerKey
1138,TIMSS_1995_8_Q14,A mixture of powdered iron and sulfur is heate...,"{'text': ['a single element', 'two other eleme...",E
1941,TIMSS_2003_4_pg36,Large amounts of metals like iron and aluminum...,"{'text': ['dead trees', 'water', 'animal bones...",E
3692,TIMSS_1995_8_P4,What happens when an animal hibernates?,{'text': ['There is no life in any of its part...,E


In [13]:
df = df[df['answerKey'] != 'E']
filtered_rows = df[df['answerKey'] == 'E']
filtered_rows

,id,question,choices,answerKey


In [14]:
df.dtypes

id           object
question     object
choices      object
answerKey    object
dtype: object

In [15]:
def make_into_prompt(row):
  cop = 'Nothing'
  if row['answerKey'] == '1' or row['answerKey'] == 1:
      cop = 'A'
  elif row['answerKey'] == '2' or row['answerKey'] == 2:
      cop = 'B'
  elif row['answerKey'] == '3' or row['answerKey'] == 3:
      cop = 'C'
  elif row['answerKey'] == '4' or row['answerKey'] == 4:
      cop = 'D'
  else:
      cop = row['answerKey']
  choices = row['choices']
  op_a = choices['text'][0] if len(choices['text'])>=1 else 'Nothing'
  op_b = choices['text'][1] if len(choices['text'])>=2 else 'Nothing'
  op_c = choices['text'][2] if len(choices['text'])>=3 else 'Nothing'
  op_d = choices['text'][3] if len(choices['text'])>=4 else 'Nothing'

  question = '{}\nOptions:\nA. {}\nB. {}\nC. {}\nD. {}\n'.format(row['question'], op_a, op_b, op_c, op_d)
  answer = cop
  prompt = f"""
    Question:
    {question}
    [INST] Solve this Multiple Choice Question and provide the correct option out of four options(A,B,C,D) as the answer. [/INST]
    Answer: {answer} </s>"""

  #print('Prompt:',prompt)
  return prompt

df['prompt'] = df.apply(make_into_prompt, axis=1)
df.to_csv('AI2_ARC_MCQA_DATASET_WITH_PROMPTS.csv',index=False)

In [16]:
pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-85vwxock/unsloth_bd37aef4dada4b7ca47bbfeaa16df9e2
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-85vwxock/unsloth_bd37aef4dada4b7ca47bbfeaa16df9e2
  Resolved https://github.com/unslothai/unsloth.git to commit 72aee4a3a128268c3d1e5cef249626897996645c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 939.8 kB/s eta 0:00:00
  Created wheel for unsloth: filename=unsloth-2024.4-py3-none-any.whl size=99740 sha256=e205cd69cdf3880c90fff906aca5123f3063e0a02853ae11a5b1a753b494f5fb
  Stored in directory: /tmp/pip-ephem-wheel-cache-5cvd9e5l/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [17]:
pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.7 MB/s eta 0:00:00


1145

In [19]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

In [20]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [35]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device("cuda")
    print("GPU is available.")
else:
    # If GPU is not available, fall back to CPU
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

model = model.to(device)

GPU is available.


In [26]:
from datasets import Dataset
final_dataset = Dataset.from_pandas(df)
final_dataset

Dataset({
    features: ['id', 'question', 'choices', 'answerKey', 'prompt', '__index_level_0__'],
    num_rows: 5194
})

In [30]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_dataset,
    dataset_text_field = "prompt",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        #max_steps = None,
        num_train_epochs = 1,# Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "answerKey",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/5194 [00:00<?, ? examples/s]

In [31]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,194 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 649
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.903600
2,0.926900
3,0.801900
4,0.812500
5,0.779900
6,0.772700
7,0.824300
8,0.856300
9,0.825700
10,0.734100


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [40]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    "Which is the best football club in the World? 12. Liverpool 23. Real Madrid . Manchester City D. Barcelona"
], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['<s> Which is the best football club in the World? A. Liverpool B. Real Madrid C. Manchester City D. Barcelona\n\nAnswer: D </s>']

In [41]:
model.save_pretrained('ai2_arc_instruction_tuned_mistral_7b')
tokenizer.save_pretrained("ai2_arc_instruction_tuned_mistral_7b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('ai2_arc_instruction_tuned_mistral_7b/tokenizer_config.json',
 'ai2_arc_instruction_tuned_mistral_7b/special_tokens_map.json',
 'ai2_arc_instruction_tuned_mistral_7b/tokenizer.model',
 'ai2_arc_instruction_tuned_mistral_7b/added_tokens.json',
 'ai2_arc_instruction_tuned_mistral_7b/tokenizer.json')

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [44]:
cd /


/


In [48]:
!zip -r model_mistral_7b_mcq.zip /content/ai2_arc_instruction_tuned_mistral_7b

  adding: content/ai2_arc_instruction_tuned_mistral_7b/ (stored 0%)
  adding: content/ai2_arc_instruction_tuned_mistral_7b/special_tokens_map.json (deflated 79%)
  adding: content/ai2_arc_instruction_tuned_mistral_7b/tokenizer.model (deflated 55%)
  adding: content/ai2_arc_instruction_tuned_mistral_7b/adapter_model.safetensors (deflated 7%)
  adding: content/ai2_arc_instruction_tuned_mistral_7b/adapter_config.json (deflated 53%)
  adding: content/ai2_arc_instruction_tuned_mistral_7b/README.md (deflated 66%)
  adding: content/ai2_arc_instruction_tuned_mistral_7b/tokenizer.json (deflated 74%)
  adding: content/ai2_arc_instruction_tuned_mistral_7b/tokenizer_config.json (deflated 64%)


In [51]:
!zip -r answerKey.zip /content/answerKey

updating: content/answerKey/ (stored 0%)
updating: content/answerKey/checkpoint-500/ (stored 0%)
updating: content/answerKey/checkpoint-500/special_tokens_map.json (deflated 79%)
updating: content/answerKey/checkpoint-500/tokenizer.model (deflated 55%)
updating: content/answerKey/checkpoint-500/rng_state.pth (deflated 25%)
updating: content/answerKey/checkpoint-500/adapter_model.safetensors (deflated 7%)
updating: content/answerKey/checkpoint-500/adapter_config.json (deflated 53%)
updating: content/answerKey/checkpoint-500/README.md (deflated 66%)
updating: content/answerKey/checkpoint-500/scheduler.pt (deflated 56%)
updating: content/answerKey/checkpoint-500/tokenizer.json (deflated 74%)
updating: content/answerKey/checkpoint-500/optimizer.pt (deflated 17%)
updating: content/answerKey/checkpoint-500/training_args.bin (deflated 51%)
updating: content/answerKey/checkpoint-500/tokenizer_config.json (deflated 64%)
updating: content/answerKey/checkpoint-500/trainer_state.json (deflated 82%

In [53]:
!zip -r hf_tokenizers_cache.zip /content/huggingface_tokenizers_cache

updating: content/huggingface_tokenizers_cache/ (stored 0%)
updating: content/huggingface_tokenizers_cache/models--unsloth--mistral-7b-instruct-v0.2-bnb-4bit/ (stored 0%)
updating: content/huggingface_tokenizers_cache/models--unsloth--mistral-7b-instruct-v0.2-bnb-4bit/snapshots/ (stored 0%)
updating: content/huggingface_tokenizers_cache/models--unsloth--mistral-7b-instruct-v0.2-bnb-4bit/snapshots/2bde61d36dbcfb34a485b073ca87615c2bd032a5/ (stored 0%)
updating: content/huggingface_tokenizers_cache/models--unsloth--mistral-7b-instruct-v0.2-bnb-4bit/snapshots/2bde61d36dbcfb34a485b073ca87615c2bd032a5/special_tokens_map.json (deflated 73%)
updating: content/huggingface_tokenizers_cache/models--unsloth--mistral-7b-instruct-v0.2-bnb-4bit/snapshots/2bde61d36dbcfb34a485b073ca87615c2bd032a5/tokenizer.model (deflated 55%)
updating: content/huggingface_tokenizers_cache/models--unsloth--mistral-7b-instruct-v0.2-bnb-4bit/snapshots/2bde61d36dbcfb34a485b073ca87615c2bd032a5/tokenizer.json (deflated 74%)